# PyTorch workflow

Let's explore an example PyTorch end-to-end workflow.

Resources:
* [Groud truth notebook](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/01_pytorch_workflow.ipynb)
* [Book version of notebook](https://www.learnpytorch.io/01_pytorch_workflow/)

In [ ]:
import torch
from torch import nn  # nn contains all of pytorch's building blocks for neural networks
import matplotlib.pyplot as plt

# Check PyTorch version
torch.__version__

# 1. Data (preparing and loading)

Data can ve almost anything... in ML
* Excel spreadsheet
* Images
* Videos
* Audio
* DNA
* Text

ML is a game of 2 parts:
1. Get data into numerical representation
2. Build a model to learn patterns in that numerical representation

To showcase this, let's create some *known* data using linear regression formula.

We'll use a linear regression formula to make a straight line with *known* **parameters**.

In [ ]:
# Create *known* parameters
weight = 0.7
bias = 0.3

# Create
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=-1)
y = weight * X + bias

X[:10]

In [ ]:
len(X), len(y)

## Splitting data into training and tests sets

In [ ]:
# Create a train/test split

train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

Visualizing the data

In [ ]:
def plot_predictions(train_data=X_train,
                     train_labels=y_train,
                     test_data=X_test,
                     test_labels=y_test,
                     predictions=None):
    """
    Plots training data, test data and compares predictions
    """
    plt.figure(figsize=(10, 7))

    # Plot training data in blue
    plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")

    # Plot testing data in green
    plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

    if predictions is not None:
        plt.scatter(test_data, predictions, c='r', labels="Predictions")

    plt.legend(prop={'size': 14})
    plt.show()

In [ ]:
plot_predictions()